# G3 Conversion

We put off doing the G3 upgrade to the KE5RCS repeater for far too long. What pushed us over the edge was a hardware failure that took out our RP2C controller unit. When we looked into repairing the device, we discovered that ICOM has basically abandoned these controllers and moved on to new repeater hardware. After considering options, we elected to buy a single ID-RP4010V repeater module and retire the full stack of repeaters we still have at the repeater site. Those may be surplussed at a later time.

So, with new hardware in hand, we decided it was time to upgrade to G3. As part of that decision, we also elected to purchase a used Dell computer that was much more current than the old gateway server.  

I did a bit of research into the upgrade process and found an overview of D-Star and the G3 conversion on Ve3IPS's blog {cite}`ve3ips2017`. See [G3 Upgrade](https://ve3ips.wordpress.com/2017/03/23/ve3yyz-upgraded-to-g3-d-star/) for more background.

## Backup User Data from G2 Server

With the old gateway machine still running, we followed these steps to create a backup for the new system. Note that we are not touching the disk in that machine until the new gateway system is fully operational, ust in case it is needed later.

As root, use these commands to create the backup:

```{shell}
cd /tmp
curl -O dsync.dstar.info/G3/g2_upgrade backup
chmod 755 g2_upgrade_backup
./g2_upgrade_backup
shutdown -h now
```

This script offers to save your backup on a remote server. I elected not to send a copy to that backup to that server. Once the new gateway  machine is running, I will use **scp** to copy the file to the new machine. Here is the command that copie the backup file to my laptop:

From my Macbook:

```{shell}
scp n5ujh@192.168.1.28:/home/n5ujh/g2_backup.tar g2_backup.tar
```

## Building the CentOS 7 Basic gateway server.

I downloaded the CentOS 7 ISO file for the basic GUI version of the server. The instructions for setting up a gateway machine call for using the minimal version of the OS, but that means that managing the machine must be done using the command line exclusively. I have found it easier to use the GUI for on-site management tasks, plus my co-administrator, is not all that comfortable using the command line.

### Network Setup

When I put together the G2 gateway machine many years ago, I created a second LAN inside my home network using a second router that would end up beinginstalled at the repeater site. The WAN port on the second router was connected to an available Ethernet port on my primary home router so that the second router could get an IP address using DHCP from the primary router.

For the G3 setup, we used an ASUS RT-N12D1 router to recreate this setup. With the second router powered up, I used the ASUS Router app in my iPhone to find the second router, then I was able to set up basic administrator access for the router.

I configured the LAN side of the ASUS router with an address of 10.0.0.1 as required by ICOM. Once that was configured and applied to the router, I was able to connect to the ASUS router over Wifi and access the administrator control panel as usual. 


I used Balena Etcher on my Macbook to create a bootable USB drive and set the the new machine to boot from the CD. 

## Step 1: Create a Bootable USB

The new machine we purchased a older Dell Optiplex machine with 4GB Ram, 128GB SSD, a DVD, and two ethernet adapters to replace our old HP machine. Unfortunately, the DVD was not working, so I used *Etcher* to build a bootable USB Flash image from the Centos 7 iso. We elected to install the DVD edition of the OS, since managing the machine is easier for non-expert Linux administrators. With this USB, we set the BIOS (hit F2 on the Dell when booting) to boot from DVD, the SSD, then USB. 

## Install a Fresh SSD in Gateway

We installed a Samsung 250GB SSD drive in the Dell machine. This is overkill for this application, but these drives are getting pretty cheap, and it was available.

## Boot the Gateway Machine

With a new SSD drive in the machine, we plugged in the USB drive and powered up the Dell. The system booted into the CentOS installer and from that point, we followed the ICOM provided guide to configuring the machine.

## ICOM Setup steps

These steps are fairly simple. 

- Select Language: English
- Set Date & Time: Chicago (could not set Network Time here)
- Set Destination Drive: SSD (Had to work through the reclaim space to delete old partitions)
- configure network as indicated in the setup guide
- Set Root Password: (rb set)
- Create User: n5ujh (administrator)

Once these settings were in place, the OS installation proceeded.

When complete, pull the USB and reboot (had to power down)
    

With the new machine running in my home LAN, I copied the G2 backup to the new machine:

```{shell}
scp g2_backup.tar n5ujh@192.168.1.28:/home/n5ujh/
```

TheGUI did not come up the first time I booted he system. That can be fixed later.

## Verify Administrator Can Log in

The system boots to a shell logon prompt. I logged in using the credentials set, and verified that I could access SUDO commands using th eroot password set. 

## Add GUI Components

The G3 setup instructions call for the "minimal" version of the OS. Hoever, that means that all work on the gateway server will be done on the command line. In our situation, the repeater is administered remotely for gateway software issues, and locally for radio issues. The local administrator is not really comfortable running commands from the command line, so we elected to install the graphical version of the OS. 

The basic installation of the OS did not load the GUI administration tools. Once the system was running, I ran these commands to install them:

```{shell}
sudo su
yum groupinstall "GNOME Desktop" "Graphical Administration Tools" -y
systemctl set-default graphical.target
/sbin/reboot
```

When the GUI starts up the first time, there is a bit of configuring left to do.

## Transfer G3 Software

I could not convince the OS to mount a Fat32 formatted USB drive, so I copied the files using **scp**:

```{shell[
scp dstar_gw-3.10-8.el7.x86_64.rmp n5ujh@192.168.1.28:/home/n5ujh
scp dstar_gw_gpg-3.10-3.noarch.rpm n5ujh@192.168.1.28:/home/n5ujh
scp install-310.sh n5ujh@192.168.1.28:/home/n5ujh
```


## Update the System

```{shell}
yum -y update
/sbin/reboot
```

This got the system ready for the G3 software installation. Before that is possible, the router needs to be configured with all port-forwarding settings needed, since the installation process checks that.


## Router Port Forwarding

The Asus router needs to be configured with these port forwarding 
settings:

- 20001-20005: TCP,UDP to 10.0.0.2 (data sync.dplus - dsipsvd.conf)
- 40000: UDP to 10.0.0.2 (voice data transfer - dsgwd.conf)
- 40001: TCP to 10.0.0.2 (data transfer dsgwd.conf)
- 443: TCP to 10.0.0.2 (web interface)
- 12345: UDP to 10.0.0.2 (position request - dsipsvd.conf)
- 12346: UDP to 10.0.0.2 (position registration - dsipsvd.conf)

## Run g3_installer Script

- Enter CallSign: KE5RCS
- Enter Backup Keyword: N5UJH
- Enter repeater latitude: 30.7333
- Enter repeater longitude: -97.5793 (from Google Maps)
- Enter Module B: 440.575 +5.00 MHz
- Enter coverage area: 40 miles

The script is supposed to use the G2 backup file and restore tables in the Postgres database. However, in repeated runs, it does not even try to do this. I decided to unpack this script and see if I can figure out why!

Let's download the installer script from the server:    

In [1]:
!scp n5ujh@ke5rcs:G3/g3_installer .

g3_installer                                  100%   19KB 132.1KB/s   00:00    


In [2]:
!ls

g3-conversion.ipynb g3_installer


This script is substantially shorter than the G2 script. In the following sections, I will break the script apart and see what major steps it performs.

The first part of the script defines variables for use later:

```{shell}
VERSION=3.4.3
# 
RECONFIG=false
SKIP_PORTTEST=0
export GW_IP=10.0.0.1
export ZR_IP=172.16.0.1
#
KERNEL=`/bin/uname -r`;
CURL="/usr/bin/curl --connect-timeout 120 --max-time 360"
ROOTG3=rootg3.dstarusers.org
BACKDIR=/dstar/backups
#
# ICOM G3
#
DSTARGW=/opt/products/dstar/dstar_gw
DSIPSVD=$DSTARGW/dsipsvd/dsipsvd
DSGWD=$DSTARGW/dsgwd/dsgwd
DSGWDCONF=$DSTARGW/dsgwd/dsgwd.conf
DBDELREC=$DSTARGW/dsipsvd/dbdelrec
DSICONF=$DSTARGW/dsipsvd/dsipsvd.conf
#
# DSync 
#
DSYNC=https://dsync.dstarusers.org
UPDATE_URL=$DSYNC/dsync.php
SCR_PATH=/dstar/scripts
SCHED=gw_schedule_g310
UPDATER=update_scripts_g310
#
# AA4RC Utils
#
TOOLS=/dstar/tools
DPLUSCONF=$TOOLS/dplus.conf
IPPORTTEST=IPPortTestClient.bin
OPENDSTAR=http://www.opendstar.org/tools
#
# DStarMonitor 
#
DSM_DIR=/opt/dstarmon
DSM_BAK=/opt/dstarmon-`date +%Y%m%d-%H%M%S`
DSM_PROP=dstarmonitor.properties
DSM_CONF=$DSM_DIR/$DSM_PROP
DSM_INIT=dsm4_init
DSMTAR=dstarmonitor43.tar.gz
#
# Postgres
# 
PG_URL=https://download.postgresql.org/pub/repos/yum/reporpms/EL-7-x86_64/pgdg-redhat-repo-latest.noarch.rpm
#
```

Next, the script checks to verify that it is being run as **root**:

```{shell}
if [ $EUID != 0 ] ; then
    echo "Installation script must be run as root.  Aborting!" | tee -a $LOGF
    exit
fi
```

You might see a problem here. The **$LOGF** variable, used here has not defined. This is not a big deal, since all that happens is that the root error message will not end up in the log file.


This next block of code checks to see where the script is running, and gets the current date and time. Then it looks to see if the **g3.zip** file is available.

```{shell}
CWD=`/bin/pwd`
DATE=`date "+%y%m%d-%H%M%S"`
LOGF=/tmp/G3-Installation-$DATE.log
```
G3ZIP=`find /tmp -iname g3.zip`


This next block checks the command line ot see what options the user selected.

```{shell]
# 
# Parse command line arguments
#

while [[ $# -gt 0 ]]; do
    argv="$1"
    case $argv in
        -n|--noporttest)
	    SKIP_PORTTEST=true
	    ;;
        -d|--debug)
	    DEBUG=true
	    ;;
        -g|--gwip)
	    GW_IP=$2
	    shift
	    ;;
        -r|--reconfigure)
	    RECONFIG=true
	    ;;
        -z|--zrip)
	    export ZR_IP=$2
	    shift
	    ;;
	*)
	    echo Unknown Option: $argv
	    echo -e "\n\t Usage: $0 -n|--noporttest -d|--debug -g|--gwip {router IP} -z|--zrip {rp2c IP} -r|--reconfigure\n"
	    exit
	    ;;
    esac
    shift
done
```

That last "Usage" section shows what options are available. 

- **-n**: skips the port test section (**SKIP_PORTTEST**)
- **-d**: enables debug messages (**DEBUG**)
- **-g**: set the gateway IP address (**GW_IP**)
- **-z**: set the rp2c (controller) IP addresses. (**ZR_IP**)
- **-r**: reconfigure the gateway. (**RECONFIGURE**)

We display a sign-on message and start the log file:
    
    ```{shell}
    #
echo `date` "ICOM G3 Gateway Upgrade Installer, Version = $VERSION"
echo `date` "$0 Starting, Log file = $LOGF" | tee -a $LOGF
```

The script is supposed to be run from the **/tmp** directory. This section makes sure that is true:

```{shell}

if [ "$CWD" != "/tmp" ]; then
    cd /tmp
fi
```

This next chunk tests to see if the **g3.zip** file was found. If so, it installs **unzip** and unpacks the file to get the two installation **rpm** files.

The outer **if** checks the **RECONFIG** variable. If it is **false**, the **rpm** files will be installed. If it is **true** the script skips installing these files.

```{shell}
#
# Make sure we can find the ICOM RPMs directly, or the G3.zip archive
#

if [ "$RECONFIG" != "true" ]; then

    if [ ! -z $G3ZIP ]; then
	if [ -f $G3ZIP ]; then
	    echo `date` "Found G3.zip Archive" | tee -a $LOGF
	    echo `date` "Installing \"unzip\" utility and unzipping..." | tee -a $LOGF
	    /bin/yum -y install unzip >> $LOGF 2>&1    
	    if [ "$?" != "0" ]; then
		echo `date` "Error installing unzip" | tee -a $LOGF
	    fi
	    /bin/unzip -o $G3ZIP >> $LOGF 2>&1
	    if [ "$?" != "0" ]; then
		echo `date` "Error unzipping $G3ZIP archive." | tee -a $LOGF
	    fi
	fi	
    else
	echo "G3 ZIP archive not found, looking for G3 RPMs" | tee -a $LOGF
    fi
```

At this point, the **rpm** files should be available. The script makes sure they are available:

```{shell}
    GW3RPM=`find . -name dstar_gw-\*.el7.x86_64.rpm`
    GWGPG=`ls -1 dstar_gw_gpg*.noarch.rpm`

    if [ -z $GW3RPM ]; then
	echo "ICOM G3 RPM not found, Aborting!"
	exit
    fi
    
    if [ -z $GWGPG ]; then
	echo "ICOM PGP Key RPM not found, Aborting!"
	exit
    fi
fi
```

If we get here, we are ready to install the **rpm** files.

Next we define functions that start and stop the gateway services:

```{shell}
#
# functions
# 

function stop () {
    /bin/systemctl stop dstar_gw
    /bin/systemctl stop dplus
    /bin/systemctl stop tomcat
    /bin/systemctl stop dsm
}

function start () {
    /bin/systemctl start dstar_gw
    /bin/systemctl start dplus
    /bin/systemctl start tomcat
    /bin/systemctl start dsm
}
```

This next unction reads the keyboard looking for a "key=val"string. It strips any white space then sets the two variables **\$VAR** and **\$FILE**

```{shell}
function key_val () {
    VAR=$1
    FILE=$2
    local VAL=`egrep -e ^$VAR $FILE | sed -e 's/ //g' | awk -F"=" '{print $2}' |sed -e 's/\r//'`;
    echo $VAL
}
```

The next chunk installs tools under **/dstar/tools**:

```{shell}
#
# Create /dstar/tools directory
# 
mkdir -p $TOOLS
```

Next, download the port test code from **opendstar.org/tools**:

```{shell}
echo -n `date` "Downloading IP Port Test Utility ... " | tee -a $LOGF
$CURL $OPENDSTAR/G3/$IPPORTTEST > $TOOLS/$IPPORTTEST 2>/dev/null
echo "DONE"

if [ "$?" != "0" ]; then
    echo "Warning - Unable to $CURL $OPENDSTAR/G3/$IPPORTTEST" | tee -a $LOGF
else
    chmod 755 $TOOLS/$IPPORTTEST
fi
```

This block installs a number of packages:

```{shell}
echo `date` "Installing required packages " | tee -a $LOGF

PACKAGES="policycoreutils-python glib2.i686 ntp net-tools openssl098e.i686 openssl-libs.i686 libpcap postgresql-libs.i686 wget"
/bin/yum -y install $PACKAGES >> $LOGF 2>&1

if [ "$?" != "0" ]; then
    echo `date` "## ERROR - Unable to install $PACKAGES with yum, aborting." | tee -a $LOGF
    exit
fi
```


Configure the Time Service

```{shell}
# 
# Start ntp time service
#
systemctl enable ntpd.service | tee -a $LOGF
```

Now we start the installation of the **rpm** files. The services need to be stopped before this happens

```{shell}

if [ "$RECONFIG" == "true" ]; then
    echo `date` "Stopping DSTAR daemons before re-configuration "
    stop
fi
```

Run the port test unless we are skipping (**-r**):

```{shell}
if [ $SKIP_PORTTEST != "true" ]; then
    echo `date` "Running IP port test to validate router port forwarding configuration:"  | tee -a $LOGF

    if [ -x $TOOLS/$IPPORTTEST ]; then
	echo `date` "Stopping firewall for port forwarding test" | tee -a $LOGF
	systemctl stop firewalld | tee -a $LOGF

	$TOOLS/$IPPORTTEST >& /tmp/porttest.log
	if grep --quiet FAILED /tmp/porttest.log; then
	    echo `date` "FAILED IP Port forwarding test!" | tee -a $LOGF
	    echo -e "\n\t#### You MUST correct the following port forwarding problem before your gateway will be functional: ####\n" | tee -a $LOGF
	    cat /tmp/porttest.log
	    echo | tee -a $LOGF
	    /bin/stty sane
	else
	    echo `date` "Gateway router has passed the port forwarding test" | tee -a $LOGF
	fi
    else
	echo `date` "$TOOLS/$IPPORTTEST not installed, or not executable!" | tee -a $LOGF
    fi
else
    echo `date` "Skipping IP Port Test" | tee -a $LOGF
fi
```

Next, install the **rp2c** utility:

```{shell}
#
# Install rp2c utility
#

$CURL $OPENDSTAR/G2/rp2c > $TOOLS/rp2c 2>/dev/null
if [ "$?" != "0" ]; then
    echo "Warning - Unable to $CURL $OPENDSTAR/G2/rp2c utility"
else
    chmod 755 $TOOLS/rp2c
    echo `date` "Please confirm your rp2c configuration is correct" | tee -a $LOGF
    $TOOLS/rp2c -a -i $ZR_IP 2>/dev/null | tee -a $LOGF
fi
```

Running this program manually indicates that the controller is not responding. It is unclear if the new ID-R4010V repeater is responding to the same commands as the old controller.


Now the script installs the gateway **gpg** **rpm** file:

```{shell}
#
# Install the ICOM Software
#

if [ "$RECONFIG" != "true" ]; then

    echo `date` Installing ICOM G3 Gateway Software | tee -a $LOGF

    echo `date` Installing $GWGPG | tee -a $LOGF
    /bin/rpm --force -ivh $GWGPG >> $LOGF 2>&1
   
    if [ "$?" != "0" ]; then
	echo "Error installing $GWGPG, check $LOGF for more information" | tee -a $LOGF
	exit
    fi
# 
# Now remove the package, leaving the ICOM GPG key
#
    GWGPG_RPM=`echo $GWGPG | sed s/.rpm//`

    echo `date` "Removing $GWGPG_RPM ... " | tee -a $LOGF
    /bin/rpm -e $GWGPG_RPM >> $LOGF 2>&1


```

The Postgres database program is installed next:
    
```{shell}
#
# Install Posgres
#
    echo `date` "Installing Postgres" | tee -a $LOGF

    yum -y localinstall $PG_URL >> $LOGF 2>&1
	if [ "$?" != "0" ]; then
	    echo "Error installing Postgres, check $LOGF for more information" | tee -a $LOGF
	    exit
	fi
```

Finally, the actual G3 software is installed:
    
```{shell}
#
# Install G3 RPM
#
    echo `date` "Installing $GW3RPM (this can take up to 5 minutes)"  | tee -a $LOGF
    /bin/yum -y localinstall $GW3RPM >> $LOGF 2>&1 
    
    if [ "$?" != "0" ]; then
	echo "Error installing $GW3RPM, check $LOGF for more information" | tee -a $LOGF
	exit
    fi
fi
```

The repeater call sign is installed in **dsipsvd.conf**

```{shell}
#
# Update ZR_CALLSIGN in dsipsvd.conf
# 
CUR_ZR_CALLSIGN=$(key_val ZR_CALLSIGN $DSICONF)
ZR_CALLSIGN=""

while [ -z $ZR_CALLSIGN ]; do
    echo 
    echo -n "Please enter gateway callsign, e.g. AB9CDE (enter Q to quit): [$CUR_ZR_CALLSIGN] " | tee -a $LOGF
    read ZR_CALLSIGN
    echo | tee -a $LOGF

    if [ "$ZR_CALLSIGN" == "Q" ] || [ "$ZR_CALLSIGN" == "q" ]; then
        exit
    elif [ -z $ZR_CALLSIGN ] && [ ! -z $CUR_ZR_CALLSIGN ]; then
        ZR_CALLSIGN=$CUR_ZR_CALLSIGN
    elif [ -z $ZR_CALLSIGN ]; then
        echo | tee -a $LOGF
        echo " Gateway Call Sign MUST be specified!" | tee -a $LOGF
    fi
done

```

The script keeps asking until the call sign is entered, or "Q", which stops the script

```{shell}
ZR_CALLSIGN=`echo $ZR_CALLSIGN | tr [a-z] [A-Z]`
echo `date` "Setting ZR_CALLSIGN = $ZR_CALLSIGN in $DSICONF" | tee -a $LOGF
/bin/sed -i -e "s/^ZR_CALLSIGN.*=.*/ZR_CALLSIGN=$ZR_CALLSIGN/" $DSICONF
```

The trust server is configured into the **dsipsvd.conf** file:
    
```{shell}
#
# Configure TRUST_SERVER in dsipsvd.conf
# 

TRUST_SERVER=$(key_val TRUST_SERVER $DSICONF)
TRUST_SERVER=`echo $TRUST_SERVER | tr [A-Z] [a-z]` | tee -a $LOGF

if [ "$TRUST_SERVER" != "$ROOTG3" ]; then
    echo `date` "Setting TRUST_SERVER = $ROOTG3 in $DSICONF" | tee -a $LOGF
    /bin/sed -i -e "s/^TRUST_SERVER.*=.*/TRUST_SERVER=$ROOTG3/" $DSICONF
else
    echo `date` "Valid TRUST_SERVER: $TRUST_SERVER" | tee -a $LOGF
fi
```

```{shell}
#
# update dsgwd.conf
#
/bin/ping -c 2 $GW_IP >> $LOGF 2>&1
if [ "$?" != "0" ]; then
    echo "Network interface at $GW_IP appears to be down, Aborting Installation!" | tee -a $LOGF
    exit
fi

GW_ETHIFNAME=`/usr/sbin/ip neigh | grep $GW_IP | head -1 | awk '{print $3}'`
GW_IFCFG=$(key_val GW_ETHIFNAME $DSGWDCONF)

if [ "$GW_IFCFG" != "$GW_ETHIFNAME" ]; then
    echo `date` "Setting GW_ETHIFNAME=$GW_ETHIFNAME" | tee -a $LOGF
    /bin/sed -i -e "s/^GW_ETHIFNAME.*=.*/GW_ETHIFNAME=$GW_ETHIFNAME/" $DSGWDCONF
else
    echo `date` "Valid GW_ETHINFNAME: $GW_IFCFG" | tee -a $LOGF
fi

ZR_ADDR=$(key_val ZR_ADDR $DSGWDCONF)

if [ "$ZR_ADDR" != "$ZR_IP" ]; then
    echo `date` "Setting ZR_ADDR = $ZR_IP in $DSGWDCONF" | tee -a $LOGF
    /bin/sed -i -e "s/^ZR_ADDR.*=.*/ZR_ADDR=$ZR_IP/" $DSGWDCONF
else
    echo `date` "Valid ZR_ADDR: $ZR_ADDR" | tee -a $LOGF
fi

DNS_ADDR=$(key_val DNS_ADDR $DSGWDCONF)

if [ "$DNS_ADDR" != "$GW_IP" ]; then
    echo `date` "Setting DNS_ADDR = $GW_IP in $DSGWDCONF" | tee -a $LOGF
    /bin/sed -i -e "s/DNS_ADDR.*=.*/DNS_ADDR=$GW_IP/" $DSGWDCONF
else
    echo `date` "Valid DNS_ADDR: $DNS_ADDR" | tee -a $LOGF
fi
```

```{shell}
# DNS_MAC in dsgwd.conf
#

DNS_MAC=$(key_val DNS_MAC $DSGWDCONF)

ARP_ENTRY=`/sbin/ip neigh | grep $GW_IP | head -1 | awk '{print $5}' | sed s/:/-/g`
LEN=${#ARP_ENTRY}

if [ -z $ARP_ENTRY ] || [ "$LEN" != "17" ]; then
    echo `date` "Unable to determine MAC address for $GW_IP, aborting!" | tee -a $LOGF
    exit
fi

if [ "$DNS_MAC" != "$ARP_ENTRY" ]; then
    echo `date` "Setting DNS_MAC = $ARP_ENTRY in $DSGWDCONF" | tee -a $LOGF
    /bin/sed -i -e "s/DNS_MAC.*=.*/DNS_MAC=$ARP_ENTRY/" $DSGWDCONF
else
    echo `date` "Valid DNS_MAC: $DNS_MAC = $ARP_ENTRY" | tee -a $LOGF
fi
```

```{shell}
#
# ZR_CALLSIGN in dsgwd.conf
# 
echo `date` "Setting DSGWD ZR_CALLSIGN = $ZR_CALLSIGN in $DSGWDCONF" | tee -a $LOGF
/bin/sed -i -e "s/^ZR_CALLSIGN.*=.*/ZR_CALLSIGN=$ZR_CALLSIGN/" $DSGWDCONF
```

```{shell}
#
# Install gateway scheduler script
#

echo `date` "Installing gw_schedule and updater scripts" | tee -a $LOGF
mkdir -p $SCR_PATH

$CURL $DSYNC/$SCHED > $SCR_PATH/$SCHED 2>/dev/null
chmod 755 $SCR_PATH/$SCHED

$CURL $DSYNC/$UPDATER > $SCR_PATH/$UPDATER 2>/dev/null
chmod 755 $SCR_PATH/$UPDATER
```


```{shell}
#
# Install new crontab
#
echo `date` "Installing DSync crontab" | tee -a $LOGF
$CURL $DSYNC/g310_dsync_crontab > /etc/cron.d/dsync 2>/dev/null
```

```{shell}
# 
# Import unsync_user_mng table from backup only if it contains zero records
#

MNG_COUNT=`/bin/psql -U dstar dstar_global -c 'select count(*) from unsync_user_mng;'|grep -v 'count\|-\|row'`

if [ $MNG_COUNT == 0 ]; then
    echo `date` "MNG Table is empty, data will be restored from backup or initialized using the add_user_mng script from ICOM" | tee -a $LOGF

    if [ $NO_BACKUP == 0 ]; then
	echo `date` "Extracting unsync_user_mng data from backup file" | tee -a $LOGF
	/bin/tar -xzvf $BACKUP /dstar/backups >> $LOGF 2>&1
	UNSYNC_USER_MNG=`ls dstar/backups/unsync_user_mng*`

	if [ -f $UNSYNC_USER_MNG ]; then
	    echo `date` "unsync_user_mng table data found, loading into database..." | tee -a $LOGF
	    cp -f $UNSYNC_USER_MNG /tmp/unsync_user_mng

cat << EOS | psql -U dstar dstar_global | tee -a $LOGF
  TRUNCATE unsync_user_mng;
  VACUUM unsync_user_mng;
  COPY unsync_user_mng FROM '/tmp/unsync_user_mng' DELIMITER ',';
  \q
EOS
	else
	    echo `date` "unsync_user_mng table data not found!" | tee -a $LOGF
	fi
    fi
fi

MNG_COUNT=`/bin/psql -U dstar dstar_global -c 'select count(*) from unsync_user_mng;'|grep -v 'count\|-\|row'`
if [ $MNG_COUNT == 0 ]; then
    echo `date` "Assuming this is a new gateway installation, running add_user_mng.sh script to set up GW Administrator entry" | tee -a $LOGF
    /bin/bash $DSTARGW/dsipsvd/add_user_mng.sh  | tee -a $LOGF
else 
    echo `date` "$MNG_COUNT records found in MNG table" | tee -a $LOGF
fi
```


```{shell}
#
# Install dplus
#

echo `date` "Installing dplus  " | tee -a $LOGF

# create dv audio directory
mkdir -p /dstar/dv

# Download / run dplus installer
$CURL -O $OPENDSTAR/G2/dplus-install-g2.sh >> $LOGF 2>&1
/bin/bash ./dplus-install-g2.sh >> $LOGF 2>&1

# 
# Update dplus interfaces
#
echo `date` "Setting dplus external ethernet = $GW_ETHIFNAME in $DPLUSCONF" | tee -a $LOGF
/bin/sed -i -e "s/^externalethernet.*=.*/externalethernet=$GW_ETHIFNAME/" $DPLUSCONF

ZR_ETHIFNAME=`ip neigh | grep $ZR_IP | head -1 | awk '{print $3}'`

echo `date` "Setting dplus internal ethernet = $ZR_ETHIFNAME in $DPLUSCONF" | tee -a $LOGF
/bin/sed -i -e "s/^internalethernet.*=.*/internalethernet=$ZR_ETHIFNAME/" $DPLUSCONF
```

```{shell}
#
# Install AA4RC Tools
#
echo `date` "Installing AA4RC dstar utilities into /dstar/tools ..." | tee -a $LOGF
UTILLIST=$OPENDSTAR/G2/utillist
UTILS=`curl $UTILLIST 2>/dev/null`
cd /dstar/tools

for TOOL in $UTILS; do
   echo `date` "Downloading: $TOOL" | tee -a $LOGF
   curl -O $OPENDSTAR/G2/$TOOL 2>/dev/null
   chmod 755 $TOOL
done
```

```{shell}
#
# Install dstargateway.org certificate
#
echo `date` "Installing certificate for dstargateway.org" | tee -a $LOGF

cd /tmp
$CURL -O $OPENDSTAR/../.admins/.certs/dsgw-cert-install-G3.tgz 2>> $LOGF | tee -a $LOGF
if [ ! -f /etc/pki/tls/certs/dstargateway.org.crt ] ; then
    cp /etc/pki/tls/certs/localhost.crt /etc/pki/tls/certs/localhost.crt.pre-dstar
    cp /etc/pki/tls/private/localhost.key /etc/pki/tls/private/localhost.key.pre-dstar
fi
cd /etc/pki/tls
tar xzf /tmp/dsgw-cert-install-G3.tgz
cp /etc/pki/tls/certs/dstargateway.org.crt /etc/pki/tls/certs/localhost.crt
cp /etc/pki/tls/private/dstargatewayorg.key /etc/pki/tls/private/localhost.key
```

```{shell}
# 
# Install AE5PLs DstarMonitor Application 
#

cd /tmp

if [ -d $DSM_DIR ]; then
    echo `date` "Moving $DSM_DIR to $DSM_BAK ... " | tee -a $LOGF
    mv $DSM_DIR $DSM_BAK
fi

mkdir -p $DSM_DIR

if [ "$RECONFIG" == "true" ]; then
    echo `date` "Using $DSM_PROP from $DSM_BAK" | tee -a $LOGF
    cp -p $DSM_BAK/$DSM_PROP $DSM_DIR/.
else
    cd /tmp
    mv -f opt opt.prev >> $LOGF 2>&1
    /bin/tar -xzf $BACKUP $DSM_CONF >> $LOGF 2>&1
    if [ "$?" == "0" ]; then
	echo `date` "Copying G2 $DSM_PROP to $DSM_CONF" | tee -a $LOGF
	cp -p opt/dstarmon/$DSM_PROP $DSM_CONF
    else
	echo `date` "$DSM_PROP not found in backup, manual installation required" | tee -a $LOGF
	$CURL -O $DSYNC/G3/dsm_configurator >> $LOGF 2>&1
	$CURL $DSYNC/G3/dsm.template > $DSM_CONF 2>> $LOGF
	/bin/bash ./dsm_configurator
    fi
fi

echo `date` "Downloading DstarMonitor distribution ..." | tee -a $LOGF
curl -O $DSYNC/$DSMTAR 2>/dev/null

if [ "$?" != "0" ]; then
    echo "Failure downloading $DSMTAR" | tee -a $LOGF
fi

tar -xzvf /tmp/$DSMTAR -C $DSM_DIR >> $LOGF 2>&1

if [ -f $DSM_CONF ]; then

    echo `date` "Updating GWIntf to $ZR_ETHIFNAME" | tee -a $LOGF
    /bin/sed -i -e "s/^GWIntf.*=.*/GWIntf=$ZR_ETHIFNAME/" $DSM_CONF

    echo `date` "Updating CtrlrIP to $ZR_IP" | tee -a $LOGF
    /bin/sed -i -e "s/^CtrlrIP.*=.*/CtrlrIP=$ZR_IP/" $DSM_CONF

    echo `date` "Updating Callsign to $ZR_CALLSIGN" | tee -a $LOGF
    /bin/sed -i -e "s/^callsign.*=.*/callsign=$ZR_CALLSIGN/" $DSM_CONF

    echo `date` "Installing dsm4_init script... " | tee -a $LOGF
    $CURL $DSYNC/$DSM_INIT > /etc/init.d/dsm 2>/dev/null
    /bin/chmod 755 /etc/init.d/dsm
    /sbin/chkconfig --add dsm

    echo `date` "DStarMonitor Installation Complete" | tee -a $LOGF
else
    echo `date` "No $DSM_PROP configuration found, manual installation required" | tee -a $LOGF
    $CURL -O $DSYNC/G3/dsm_configurator >> $LOGF 2>&1
    $CURL $DSYNC/G3/dsm.template > $DSM_CONF 2>> $LOGF
    /bin/bash ./dsm_configurator
fi
```